In [14]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.datasets import make_blobs
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

**Objective and goal for this lab**

This lab is intended to give a quick user guide on how to fit an AdaBoostClassifier. Analogously, you can just as well fit an AdaBoostRegressor for suitable problems.

---

**Import the data**

In [15]:
iris_df = pd.read_csv('../data/IRIS.csv')
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [16]:
import plotly.express as px

df = px.data.iris()
fig = px.scatter_3d(iris_df, x='sepal_length', y='petal_length', z='petal_width', color='species')
fig.show()

In [17]:
iris_df['species'].value_counts()

species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

The target column *species* is categorical. We need to make it numerical by assigning each class a specific number.

In [18]:
class_map = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica': 2}

numerical_targets = [class_map[value] for value in iris_df['species']]

iris_df['species'] = numerical_targets

In [19]:
iris_df['species'].value_counts()

species
0    50
1    50
2    50
Name: count, dtype: int64

In [20]:
X, y = iris_df.drop(columns='species'), iris_df['species']

No further pre-processing or feature engineering is needed for this simple dataset.

---

**GridSearch with AdaBoost**

AdaBoost is in general composed by a sequential series of weak learners. The reason is that each one of them, by itself, will underfit - and thus have high bias. But, by using the boosting method, each weak learner can train to become good at what the previous learner was bad at. We then use all the trained learners for prediction, just like we did for bagging (in Random Forest). This will in theory eliminate the high bias problem of each individual learner, and instead give us a strong ensamble classifier/regressor.

The default weak learner when using AdaBoost is a decision tree with max depth of 1 (this is really a weak learner, right!).

If you don't specify otherwise when initializing AdaBoost, the default weak learner is thus a decision tree.

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

# define the model with default hyperparameters
model = AdaBoostClassifier()

# define the grid of values to search
param_grid = {'n_estimators': [10, 50, 100, 500],                 # antalet sekventiella 'svaga' modeller att träna
              'learning_rate': [0.001, 0.01, 0.1, 1.0]}           # avgör hur mycket varje fel i iterationen ska viktas

# define the grid search procedure
grid_search = GridSearchCV(estimator=model, 
                           param_grid=param_grid, 
                           n_jobs=-1, 
                           cv=3, 
                           scoring='accuracy')

# execute the grid search
grid_result = grid_search.fit(X, y)

# summarize the best score and configuration
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
print('---'*25, end='\n\n')

# summarize all scores that were evaluated
mean_test_scores = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean_test_score, param in zip(mean_test_scores, params):
    print('params:')
    print(f"{param}")
    print('mean accuracy:')
    print(f'{round(mean_test_score,4)}')
    print('---'*25, end='\n\n')

Best: 0.9666666666666667 using {'learning_rate': 1.0, 'n_estimators': 50}
---------------------------------------------------------------------------

params:
{'learning_rate': 0.001, 'n_estimators': 10}
mean accuracy:
0.66
---------------------------------------------------------------------------

params:
{'learning_rate': 0.001, 'n_estimators': 50}
mean accuracy:
0.66
---------------------------------------------------------------------------

params:
{'learning_rate': 0.001, 'n_estimators': 100}
mean accuracy:
0.66
---------------------------------------------------------------------------

params:
{'learning_rate': 0.001, 'n_estimators': 500}
mean accuracy:
0.7467
---------------------------------------------------------------------------

params:
{'learning_rate': 0.01, 'n_estimators': 10}
mean accuracy:
0.66
---------------------------------------------------------------------------

params:
{'learning_rate': 0.01, 'n_estimators': 50}
mean accuracy:
0.7467
----------------------

---

Above, we mentioned that AdaBoost concists of weak learners that we train sequentually, each one tries to become good at what the previous one was lacking. However, you can actually controll yourself what these weak learners should be. You can in practice choose anything to serve the role of the weak learner! In fact, it doesn't really have to be a weak learner, it can also be a strong learner!

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier


# define the model with default hyperparameters
model = AdaBoostClassifier(estimator=RandomForestClassifier())    # Note that we've here chosen RandomForestClassifier (with default hyperparameters) 
                                                                  # as the weak learner!  

# define the grid of values to search
param_grid = {'n_estimators': [10, 50, 100, 500],                 
              'learning_rate': [0.001, 0.01, 0.1, 1.0]}           

# define the grid search procedure
grid_search = GridSearchCV(estimator=model, 
                           param_grid=param_grid, 
                           n_jobs=-1, 
                           cv=3, 
                           scoring='accuracy')

# execute the grid search
grid_result = grid_search.fit(X, y)

# summarize the best score and configuration
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
print('---'*25, end='\n\n')

# summarize all scores that were evaluated
mean_test_scores = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean_test_score, param in zip(mean_test_scores, params):
    print('params:')
    print(f"{param}")
    print('mean accuracy:')
    print(f'{round(mean_test_score,4)}')
    print('---'*25, end='\n\n')

Best: 0.9666666666666667 using {'learning_rate': 0.001, 'n_estimators': 10}
---------------------------------------------------------------------------

params:
{'learning_rate': 0.001, 'n_estimators': 10}
mean accuracy:
0.9667
---------------------------------------------------------------------------

params:
{'learning_rate': 0.001, 'n_estimators': 50}
mean accuracy:
0.96
---------------------------------------------------------------------------

params:
{'learning_rate': 0.001, 'n_estimators': 100}
mean accuracy:
0.96
---------------------------------------------------------------------------

params:
{'learning_rate': 0.001, 'n_estimators': 500}
mean accuracy:
0.96
---------------------------------------------------------------------------

params:
{'learning_rate': 0.01, 'n_estimators': 10}
mean accuracy:
0.9667
---------------------------------------------------------------------------

params:
{'learning_rate': 0.01, 'n_estimators': 50}
mean accuracy:
0.96
--------------------

---

## Challenges

**Task 1**

Make sure to completely understand the whole process we've laid out above.

**Task 2**

Read more about [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) and [AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html) in respective documentation.

**Task 3**

Redo the grid search, but now include a search for the best *estimator* - in other words, do a GridSearch where you also use different models as 'weak learners'. 

Experiment away!

In [31]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

base_model = {
    "rf": RandomForestClassifier(random_state=23),
    "dt": DecisionTreeClassifier(max_depth=1, random_state=23),
    "svm": SVC(probability=True, kernel="linear", random_state=23),
    "lr": LogisticRegression(random_state=23)
}

param_grid = {"estimator": [base_model["rf"], base_model['dt'], base_model["svm"], base_model["lr"]],
              "n_estimators": [10, 50, 100],                 
              "learning_rate": [0.001, 0.01, 0.1, 1.0]}    
       
model = AdaBoostClassifier()    

grid_search = GridSearchCV(estimator=model, 
                           param_grid=param_grid, 
                           n_jobs=-1, 
                           cv=3, 
                           scoring="accuracy")

grid_result = grid_search.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
print('---'*25, end='\n\n')

mean_test_scores = grid_result.cv_results_["mean_test_score"]
params = grid_result.cv_results_["params"]
for mean_test_score, param in zip(mean_test_scores, params):
    print("params:")
    print(f"{param}")
    print("mean accuracy:")
    print(f"{round(mean_test_score,4)}")
    print("---"*25, end="\n\n")

Best: 0.9959997478738108 using {'estimator': SVC(kernel='linear', probability=True, random_state=23), 'learning_rate': 0.01, 'n_estimators': 100}
---------------------------------------------------------------------------

params:
{'estimator': RandomForestClassifier(random_state=23), 'learning_rate': 0.001, 'n_estimators': 10}
mean accuracy:
0.9935
---------------------------------------------------------------------------

params:
{'estimator': RandomForestClassifier(random_state=23), 'learning_rate': 0.001, 'n_estimators': 50}
mean accuracy:
0.9935
---------------------------------------------------------------------------

params:
{'estimator': RandomForestClassifier(random_state=23), 'learning_rate': 0.001, 'n_estimators': 100}
mean accuracy:
0.993
---------------------------------------------------------------------------

params:
{'estimator': RandomForestClassifier(random_state=23), 'learning_rate': 0.01, 'n_estimators': 10}
mean accuracy:
0.9935
-------------------------------

**Task 4**

Now, create your own (more complicated) dataset using make_blobs, as we have previously. Use the following:

n_samples = 2000

n_features = 3

n_classes = 6

Do a 3D-plot of the data to make sure you get a grasp of it. Make sure the color of each point indicate class belonging.

When that's done, also pick a random state such that the classes are somewhat difficult to distinguish.

Therafter, redo Task 3 on this new dataset.

In [29]:
n_samples = 2000
n_features = 3
n_classes = 6

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=n_classes, random_state=23)

data_dict = {"x": X[:,0], "y": X[:,1], "z": X[:,2], "class": y}
data_df = pd.DataFrame(data_dict)

df = px.data.iris()
fig = px.scatter_3d(data_df, x="x", y="y", z="z", color="class")
fig.show()

In [30]:
base_model = {
    "rf": RandomForestClassifier(random_state=23),
    "dt": DecisionTreeClassifier(max_depth=1, random_state=23),
    "svm": SVC(probability=True, kernel="linear", random_state=23),
    "lr": LogisticRegression(random_state=23)
}

param_grid = {"estimator": [base_model["rf"], base_model['dt'], base_model["svm"], base_model["lr"]],
              "n_estimators": [10, 50, 100],                 
              "learning_rate": [0.001, 0.01, 0.1, 1.0]}    
       
model = AdaBoostClassifier()    

grid_search = GridSearchCV(estimator=model, 
                           param_grid=param_grid, 
                           n_jobs=-1, 
                           cv=3, 
                           scoring="accuracy")

grid_result = grid_search.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
print('---'*25, end='\n\n')



Best: 0.9959997478738108 using {'estimator': SVC(kernel='linear', probability=True, random_state=23), 'learning_rate': 0.01, 'n_estimators': 100}
---------------------------------------------------------------------------

                                      param_estimator  param_n_estimators  \
29  SVC(kernel='linear', probability=True, random_...                 100   
30  SVC(kernel='linear', probability=True, random_...                  10   
24  SVC(kernel='linear', probability=True, random_...                  10   
32  SVC(kernel='linear', probability=True, random_...                 100   
26  SVC(kernel='linear', probability=True, random_...                 100   
27  SVC(kernel='linear', probability=True, random_...                  10   
28  SVC(kernel='linear', probability=True, random_...                  50   
31  SVC(kernel='linear', probability=True, random_...                  50   
25  SVC(kernel='linear', probability=True, random_...                  50   
8      